## Aditya Jhaveri : N13689134 : Assignment-5

# The SVM vs. Logistic Regression Showdown

In this lab, you will practice working with non-linear kernels combined with logistic regression and SVM classifiers. The goal is to compare these commonly used techniques. Which comes out on top in terms of accuracy? Runtime? Is there much of a difference at all?  

## Loading the Data

First, we load all the packages we'll need.

In [131]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import pairwise_kernels
import scipy
from sklearn import svm, linear_model
from sklearn.model_selection import GridSearchCV
import time
import pandas as pd

Again we download the data from the Tensorflow package, which you will need to install.  You can get the data from other sources as well.

In the Tensorflow dataset, the training and test data are represented as arrays:

     Xtr.shape = 60000 x 28 x 28
     Xts.shape = 10000 x 28 x 28
     
The test data consists of `60000` images of size `28 x 28` pixels; the test data consists of `10000` images.

In [132]:
import tensorflow as tf

(Xtr_raw,ytr),(Xts_raw,yts) = tf.keras.datasets.mnist.load_data()

print('Xtr shape: %s' % str(Xtr_raw.shape))
print('Xts shape: %s' % str(Xts_raw.shape))

ntr = Xtr_raw.shape[0]
nts = Xts_raw.shape[0]
nrow = Xtr_raw.shape[1]
ncol = Xtr_raw.shape[2]

Xtr shape: (60000, 28, 28)
Xts shape: (10000, 28, 28)


Each pixel value is from `[0,255]`.  For this lab, it will be convenient to recale the value to -1 to 1 and reshape it as a `ntr x npix` and `nts x npix`.

In [133]:
npix = nrow*ncol
Xtr = Xtr_raw.reshape((ntr,npix))
print(Xtr[1,:])
Xtr = (Xtr/255 - .5)
print(Xtr[1,:])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  51 159 253 159  50   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  48 238 252 252 252 237   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  54 227 253 252 239 233 252  57   6   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  10  60 224 252 253 252 202  84 252
 253 122   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 163 252 252 252 253 252 252  96 189 253 167   

In [134]:
npix = nrow*ncol
Xtr = (Xtr_raw/255 - 0.5)
Xtr = Xtr.reshape((ntr,npix))

Xts = (Xts_raw/255 - 0.5)
Xts = Xts.reshape((nts,npix))

In [135]:
print(Xtr)

[[-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 ...
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]]


For this lab we're only going to use a fraction of the MNIST data -- otherwise our models will take too much time and memory to run. Using only part of the training data will of course lead to worse results. Given enough computational resources and time, we would ideally be running on the full data set. The follow code creates a new test and train set, with 10000 examples for train and 5000 for test.

In [136]:
ntr1 = 10000
nts1 = 5000
Iperm = np.random.permutation(ntr1)
Xtr1 = Xtr[Iperm[:ntr1],:]
ytr1 = ytr[Iperm[:ntr1]]
Iperm = np.random.permutation(nts1)
Xts1 = Xts[Iperm[:nts1],:]
yts1 = yts[Iperm[:nts1]]

## Problem set up and establishing a baseline

To simplify the problem (and speed things up) we're also going to restrict to binary classification. In particular, let's try to design classifier a that separates the 8's from all other digits.

Create binary 0/1 label vectors `ytr8` and `yts8` which are 1 wherever `ytr1` and `yts1` equal 8, and 0 everywhere else.

In [137]:
# TODO
# ytr8 =
# yts8 =

ytr8 = np.where(ytr1 == 8, 1, 0)
yts8 = np.where(yts1 == 8, 1, 0)


Most of the digits in the test dataset aren't equal to 8. So if we simply guess 0 for every image in `Xts`, we might expect to get classification accuracy around 90%. Our goal should be to significantly beat this **baseline**.

Formally, write a few lines of code to check what test error would be achieved by the all zeros classifier.

In [138]:
# TODO

onesr8 = np.where(ytr8 == 1)
acc = (len(ytr8) - onesr8[0].shape[0])/len(ytr8)
print('Accuaracy = {0:4}'.format(acc))

Accuaracy = 0.9056


As a second baseline, let's see how we do with standard (non-kernel) logistic regression. As in the MNIST demo, you can use `scikit-learn`'s built in function `linear_model.LogisticRegression` to fit the model and compute the accuracy. Use no regularization and the `lbfgs` solver. You should acheive an improvement to around 93-95%.

In [172]:
# TODO
# ...
# acc =
# print('Logistic Regression Accuaracy = {0:f}'.format(acc))

model = linear_model.LogisticRegression(solver='lbfgs', C=1e10, max_iter=1000)
model.fit(Xtr1, ytr8)

y_pred = model.predict(Xts1)
acc = (sum(y_pred == yts8))/len(y_pred)
print('Logistic Regression Accuaracy = {0:4}'.format(acc))

Logistic Regression Accuaracy = 0.9344


## Kernel Logistic Regression

To improve on this baseline performance, let's try using the logistic regression classifier with a *non-linear* kernel. Recall from class that any non-linear kernel similarity function $k(\vec{w},\vec{z})$ is equal to $\phi(\vec{w})^T\phi(\vec{z})$ for some feature transformation $\phi$. However, we typically do not need to compute this feature tranformation explicitly: instead we can work directly with the kernel gram matrix $K \in \mathbb{R}^{n\times n}$. Recall that $K_{i,j} = k(\vec{x}_i,\vec{x}_j)$ where $\vec{x}_i$ is the $i^\text{th}$ training data point.

For this lab we will be using the radial basis function kernel. For a given scaling factor $\gamma$ this kernel is defined as:
$$
k(\vec{w},\vec{z}) = e^{-\gamma\|\vec{w}-\vec{z}\|_2^2}
$$

In [140]:
def rbf_kernel(w,z,gamma):
    d = w - z
    return np.exp(-gamma*np.sum(d*d))

Construct the kernel matrix `K1` for `Xtr1` with `gamma = .05`.

In [141]:
# TODO
gamma = .05
K1 = np.zeros((Xtr1.shape[0], Xtr1.shape[0]))
for i, row in enumerate(K1):
    for j, col in enumerate(K1.T):
        K1[i, j] = rbf_kernel(Xtr1[i, :], Xtr1[j, :], gamma)

print(K1)

[[1.00000000e+00 1.25304055e-03 1.71321421e-02 ... 2.39769750e-02
  6.70109693e-03 2.33623087e-02]
 [1.25304055e-03 1.00000000e+00 4.10431880e-03 ... 9.54983024e-04
  3.05452994e-03 6.94805035e-04]
 [1.71321421e-02 4.10431880e-03 1.00000000e+00 ... 3.03017041e-02
  8.81420192e-03 1.04144868e-02]
 ...
 [2.39769750e-02 9.54983024e-04 3.03017041e-02 ... 1.00000000e+00
  1.82182295e-02 3.82456754e-03]
 [6.70109693e-03 3.05452994e-03 8.81420192e-03 ... 1.82182295e-02
  1.00000000e+00 4.14650517e-03]
 [2.33623087e-02 6.94805035e-04 1.04144868e-02 ... 3.82456754e-03
  4.14650517e-03 1.00000000e+00]]


If you used a for loop (which is fine) your code might take several minutes to run! Part of the issue is that Python won't know to properly parallize your for loop. For this reason, when constructing kernel matrices it is often faster to us a built-in, carefully optimized function with explicit parallelization. Scikit learn provides such a function through their `metrics` library.

Referring to the documentation here
https://scikit-learn.org/stable/modules/metrics.html#metrics, use this built in function to recreate the same kernel matrix you did above. Store the result at `K`.

In [142]:
# TODO
from sklearn.metrics.pairwise import rbf_kernel as rbf_kernel_sk
K = rbf_kernel_sk(Xtr1, Xtr1, gamma=gamma)
print(K)

[[1.00000000e+00 1.25304055e-03 1.71321421e-02 ... 2.39769750e-02
  6.70109693e-03 2.33623087e-02]
 [1.25304055e-03 1.00000000e+00 4.10431880e-03 ... 9.54983024e-04
  3.05452994e-03 6.94805035e-04]
 [1.71321421e-02 4.10431880e-03 1.00000000e+00 ... 3.03017041e-02
  8.81420192e-03 1.04144868e-02]
 ...
 [2.39769750e-02 9.54983024e-04 3.03017041e-02 ... 1.00000000e+00
  1.82182295e-02 3.82456754e-03]
 [6.70109693e-03 3.05452994e-03 8.81420192e-03 ... 1.82182295e-02
  1.00000000e+00 4.14650517e-03]
 [2.33623087e-02 6.94805035e-04 1.04144868e-02 ... 3.82456754e-03
  4.14650517e-03 1.00000000e+00]]


Check that you used the function correctly by writing code to confirm that `K = K1`, or at least that the two are equal up to very small differences (which could arise due to numerical precision issues). Try to do this **without a for loop** so that the equality check takes advantage of paralellism.

In [143]:
# TODO

are_equal = np.allclose(K, K1, atol=1e-8)

print("Are the two kernel matrices equal (within a small tolerance)?", are_equal)

Are the two kernel matrices equal (within a small tolerance)? True


When using a non-linear kernel, it is important to check that you have chosen reasonable parameters (in our case the only parameter is `gamma`). We typically do not want $k(\vec{x}_i,\vec{x}_j)$ to be either negligably small, or very large for all $i\neq j$ in our data set, or we won't be able to learn anything. For the RBF kernel this means that, for any $\vec{x}_i$ we don't want $k(\vec{x}_i,\vec{x}_j)$ very close to 1 (e.g. .9999) for all $j$, or very close to $0$ (e.g. 1e-8) for all $j$.

Let's just check that we're in good shape for the first data vector $\vec{x}_0$. Do so by printing out the 10 largest and 10 smallest values of $k(\vec{x}_0,\vec{x}_j)$ for $j\neq 0$. Note that we always have $k(\vec{x}_0,\vec{x}_0) = 1$ for the RBF kernel.

In [144]:
# TODO
xK10 = K1[0]
K1_without_xK10 = np.delete(xK10, 0)

K10_largest_values = np.sort(K1_without_xK10)[-10:]
K10_smallest_values = np.sort(K1_without_xK10)[:10]

print(f"Maximum similarities: {K10_largest_values}\n")
print(f"Minimum similarities: {K10_smallest_values}\n")

Maximum similarities: [0.1441433  0.14471591 0.14665544 0.150195   0.15054071 0.15245242
 0.15304697 0.15974273 0.16782655 0.16923569]

Minimum similarities: [2.40521253e-05 2.98295584e-05 3.38176572e-05 3.77959453e-05
 3.88328604e-05 3.90582979e-05 4.27346464e-05 5.53763081e-05
 6.61567736e-05 7.14055509e-05]



### Implementation
Maybe surprisingly Scikit learn does not have an implementation for kernel logistic regression. So we have to implement our own!

Write a function function `log_fit` that minimizes the $\ell_2$-regularized logisitic regression loss:
$$
L(\boldsymbol{\alpha}) =\sum_{i=1}^n (1-y_i)(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\vec{\alpha}) - \log(h(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\boldsymbol{\alpha}) + \lambda \|\phi(\mathbf{X})^T\boldsymbol{\alpha}\|_2^2.
$$
As input it takes an $n\times n$ kernel matrix $K$ for the training data, an $n$ length vector `y` of binary class labels, and regularization parameter `lamb`.

To implement your function you can either use your own implementation of gradient descent or used a built in minimizer from `scipy.optimize.minimize`. I recommend trying the later approach. You could try using e.g. the same L-BGFG method we used earlier. In either case, you will need to write a function to compute the gradient of the logistic regression loss above. Most of the methods in `scipy.optimize.minimize` require a function that computes the gradient as input.

In [145]:
from scipy.optimize import minimize

In [169]:
def log_fit(K, y, lamb, displayBoolean=True):
    n = K.shape[0]
    alpha0 = np.zeros(n)

    def loss(alpha, K, y, lamb):
        f = K @ alpha
        h = 1 / (1 + np.exp(-f))
        return -np.sum(y * np.log(h + 1e-8) + (1 - y) * np.log(1 - h + 1e-8)) + lamb * np.dot(alpha, alpha)
    
    def gradient(alpha, K, y, lamb):
        f = K @ alpha
        h = 1 / (1 + np.exp(-f))
        grad = K.T @ (h - y) + 2 * lamb * alpha
        return grad

    result = minimize(loss, alpha0, jac=gradient, args=(K, y, lamb), method='L-BFGS-B', options=
                      {'disp': displayBoolean, 'maxiter': 20000})
    # if not result.success:
    #     print(result.x)
    #     raise ValueError("Optimization failed:", result.message)
    
    return result  # Optimal alpha

Use the `log_fit` function defined above to find parameters `alpha` for the kernel logistic regression model using `lamb = 0` and `K` as constructed above (with `gamma = .05`).

In [147]:
# TODO
lamb = 0
K = K/K.max()
result = log_fit(K, ytr8, lamb)
alpha = result.x
print(alpha)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        10000     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D+03    |proj g|=  2.13262D+02

At iterate    1    f=  4.05603D+03    |proj g|=  2.47068D+01

At iterate    2    f=  3.88698D+03    |proj g|=  2.45440D+01

At iterate    3    f=  3.34927D+03    |proj g|=  2.90947D+01

At iterate    4    f=  2.87196D+03    |proj g|=  2.81512D+01

At iterate    5    f=  2.06306D+03    |proj g|=  1.68426D+01


 This problem is unconstrained.



At iterate    6    f=  1.51593D+03    |proj g|=  7.98500D+00

At iterate    7    f=  1.22331D+03    |proj g|=  3.88072D+00

At iterate    8    f=  1.03891D+03    |proj g|=  4.77241D+00

At iterate    9    f=  8.86376D+02    |proj g|=  2.91802D+00

At iterate   10    f=  7.43484D+02    |proj g|=  3.06586D+00

At iterate   11    f=  6.23916D+02    |proj g|=  3.25988D+00

At iterate   12    f=  5.59236D+02    |proj g|=  4.14671D+00

At iterate   13    f=  5.12522D+02    |proj g|=  2.94616D+00

At iterate   14    f=  4.79588D+02    |proj g|=  1.55212D+00

At iterate   15    f=  4.64467D+02    |proj g|=  5.36346D+00

At iterate   16    f=  4.38293D+02    |proj g|=  1.89475D+00

At iterate   17    f=  4.21638D+02    |proj g|=  1.86314D+00

At iterate   18    f=  3.63711D+02    |proj g|=  1.61559D+01

At iterate   19    f=  3.13879D+02    |proj g|=  1.67204D+00

At iterate   20    f=  3.04513D+02    |proj g|=  4.68193D+00

At iterate   21    f=  2.90049D+02    |proj g|=  2.18742D+00

At iter

Suppose we have a test dataset with $m$ examples $\vec{w}_1,\ldots, \vec{w}_m$. Once we obtain a coefficient vector $\alpha$, making predictions for any $\vec{w}_j$ in the test set requires computing:
$$
{y}_{j} = \sum_{i=1}^n \alpha_i \cdot k(\vec{w}_{j}, \vec{x}_i).
$$
where $\vec{x}_1, \ldots \vec{x}_n$ are our training data vectors. We classify $\vec{w}_{j}$ in class 0 if ${y}_{j} \leq 0$ and in class 1 if ${y}_{j} > 0$.

This computation can be rewritten in matrix form as follows:
$$
\vec{y}_{test} = K_{test}\vec{\alpha},
$$
where $\vec{y}_{text}$ is an $m$ length vector and $K_{test}$ is a $m\times n$ matrix whose $(j,i)$ entry is equal to $k(\vec{w}_{j}, \vec{x}_i)$. We classify $\vec{w}_{j}$ in class 0 if $\vec{y}_{test}[j] \leq 0$ and in class 1 if $\vec{y}_{test}[j] > 0$.


Use the `pairwise_kernels` function to construct $K_{test}$. Then make predictions for the test set and evaluate the accuracy of our kernel logistic regression classifier. You should see a pretty substantial lift in accuracy to around $97\%$

In [148]:
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics import accuracy_score

In [149]:
# TODO
gamma = .05
Ktest = pairwise_kernels(Xts1, Xtr1, metric='rbf', gamma=gamma)

In [150]:
# TODO
yhat = Ktest @ alpha
y_pred = (yhat > 0).astype(int)
acc = np.mean(y_pred == yts8)
print("Test accuracy = %f" % acc)

Test accuracy = 0.979800


## Kernel Support Vector Machine

The goal of this lab is to compare Kernel Logistic Regression to Kernel Support Vector machines. Following `demo_mnist_svm.ipynb` create and train an SVM classifier on `Xtr1` and `ytr8` using an RBF kernel with `gamma = .05` (the same value we used for logistic regression above). Use margin parameter `C = 10`.

In [151]:
# TODO
# Create a SVM

from sklearn import svm

svcrbf = svm.SVC(probability=False,  kernel="rbf", C=10, gamma=.05,verbose=10)

In [152]:
svcrbf.fit(Xtr1, ytr8)

[LibSVM]....*.*
optimization finished, #iter = 5745
obj = -491.710632, rho = -0.762837
nSV = 3563, nBSV = 0
Total nSV = 3563


SVC(C=10, gamma=0.05, verbose=10)

Calculate and print the accuracy of the SVM classifier. You should obtain a similar result as for logistic regression: something close to $97\%$ accuracy.

In [153]:
# TODO
ysvm = svcrbf.predict(Xts1)
acc = np.mean(ysvm == yts8)
print("Test accuracy = %f" % acc)

Test accuracy = 0.980200


## The Showdown

Both SVM classifiers and kernel logisitic regression require tuning parameters to obtain the best possible result. In our setting we will stick with an RBF kernel (although this could be tuned). So we only consider tuning the kernel width parameter `gamma`, as well as the regularization parameter `lamb` for logistic regression, and the margin parameter `C` for SVM. We will choose parameters using for-loops and train-test cross validation.

Train a logistic regression classifier with **all combinations** of the parameters included below in vectors `gamma` and `lamb`. For each setting of parameters, compute and print:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

For computing runtime you might want to use the `time()` function from the `time` library, which we already imported ealier.

In [170]:
gammas = [.1, .05,.02,.01,.005]
lambs = [0,1e-6,1e-4,1e-2]
# TODO
# ...

dfLRC = pd.DataFrame(columns=['Gamma', 'Lambda', 'Testing Accuracy', 'Time Taken'])



for i in range(0, len(gammas)):
    gamma = gammas[i]
    for j in range(0, len(lambs)):
        lamb = lambs[j]
        print(gamma, lamb)
        K = rbf_kernel_sk(Xtr1, Xtr1, gamma=gamma)
        K = K/K.max()
        startTime = time.time()
        result = log_fit(K, ytr8, lamb, displayBoolean=False)
        print(result)
        alpha = result.x
        Ktest = pairwise_kernels(Xts1, Xtr1, metric='rbf', gamma=gamma)
        yhat = Ktest @ alpha
        y_pred = (yhat > 0).astype(int)
        acc = np.mean(y_pred == yts8)
        endTime = time.time()
        dfLRC = pd.concat([dfLRC, pd.DataFrame([[gamma, lamb, acc, endTime - startTime]], columns=dfLRC.columns)], ignore_index=True)

print("For Logistic Regression Classifier:")
dfLRC
        
        

        

0.1 0
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.002793840771287251
        x: [-1.948e+00 -1.009e+01 ... -6.306e+00 -3.442e+00]
      nit: 26
      jac: [ 4.945e-07  8.201e-07 ...  7.558e-07  3.094e-07]
     nfev: 29
     njev: 29
 hess_inv: <10000x10000 LbfgsInvHessProduct with dtype=float64>


/var/folders/3y/x88zbv4n0b18f8cg1qcv1my40000gn/T/ipykernel_39334/812275000.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfLRC = pd.concat([dfLRC, pd.DataFrame([[gamma, lamb, acc, endTime - startTime]], columns=dfLRC.columns)], ignore_index=True)


0.1 1e-06
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.368326091206752
        x: [-3.980e+00 -7.842e+00 ... -5.287e+00 -2.489e+00]
      nit: 45
      jac: [-1.558e-07 -1.892e-08 ...  6.520e-08  4.283e-09]
     nfev: 52
     njev: 52
 hess_inv: <10000x10000 LbfgsInvHessProduct with dtype=float64>
0.1 0.0001
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 16.124220382261893
        x: [-2.397e+00 -4.908e+00 ... -3.551e+00 -1.689e+00]
      nit: 51
      jac: [-3.227e-07  9.268e-08 ...  3.655e-07 -4.119e-08]
     nfev: 57
     njev: 57
 hess_inv: <10000x10000 LbfgsInvHessProduct with dtype=float64>
0.1 0.01
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 441.84140073498486
        x: [-9.788e-01 -2.205e+00 ... -1.854e+00 -9.559e-01]
      nit: 35
      jac: [ 1.175e-05  1.719e-06 ...  2.145e-06 -8.543e-07]
     nfev: 38
     njev

/var/folders/3y/x88zbv4n0b18f8cg1qcv1my40000gn/T/ipykernel_39334/3487590976.py:7: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-f))
/var/folders/3y/x88zbv4n0b18f8cg1qcv1my40000gn/T/ipykernel_39334/3487590976.py:12: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-f))


  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -8.445046415750054e-05
        x: [ 8.212e+00  4.055e+00 ... -3.769e+01  1.304e+00]
      nit: 281
      jac: [-1.023e-06 -5.067e-07 ... -6.409e-07 -7.592e-07]
     nfev: 335
     njev: 335
 hess_inv: <10000x10000 LbfgsInvHessProduct with dtype=float64>
0.02 1e-06
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.12773230262878757
        x: [-2.209e-01  8.269e-01 ... -4.242e+00  3.449e-01]
      nit: 1676
      jac: [-1.804e-06 -3.349e-06 ... -2.703e-06 -2.243e-06]
     nfev: 1992
     njev: 1992
 hess_inv: <10000x10000 LbfgsInvHessProduct with dtype=float64>
0.02 0.0001
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 5.093902427824644
        x: [ 1.526e-02  5.011e-01 ... -2.433e+00  1.787e-01]
      nit: 1985
      jac: [ 6.066e-05  1.918e-05 ...  3.502e-05  4.024e-05]
     nfev: 232

/var/folders/3y/x88zbv4n0b18f8cg1qcv1my40000gn/T/ipykernel_39334/3487590976.py:7: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-f))
/var/folders/3y/x88zbv4n0b18f8cg1qcv1my40000gn/T/ipykernel_39334/3487590976.py:12: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-f))


  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -5.7767183817373735e-05
        x: [ 2.316e+02  1.551e+02 ... -7.839e+02 -5.191e+01]
      nit: 793
      jac: [-8.172e-07  3.733e-07 ... -8.287e-07 -3.323e-06]
     nfev: 960
     njev: 960
 hess_inv: <10000x10000 LbfgsInvHessProduct with dtype=float64>
0.01 1e-06
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 0.39277482031178523
        x: [ 1.926e+00  1.228e+00 ... -7.928e+00  1.381e+00]
      nit: 7635
      jac: [-1.032e-06  5.763e-06 ... -2.281e-07 -4.300e-06]
     nfev: 9127
     njev: 9127
 hess_inv: <10000x10000 LbfgsInvHessProduct with dtype=float64>
0.01 0.0001
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 13.566761657332686
        x: [ 1.235e+00  7.922e-01 ... -4.310e+00  6.660e-01]
      nit: 6552
      jac: [ 4.975e-05  2.306e-05 ... -4.467e-05  1.112e-05]
     nfev: 7

/var/folders/3y/x88zbv4n0b18f8cg1qcv1my40000gn/T/ipykernel_39334/3487590976.py:7: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-f))
/var/folders/3y/x88zbv4n0b18f8cg1qcv1my40000gn/T/ipykernel_39334/3487590976.py:12: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-f))


  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -8.479714593384305e-05
        x: [ 1.317e+04 -2.765e+03 ... -2.097e+04  2.263e+03]
      nit: 5371
      jac: [-5.580e-06 -5.190e-06 ... -5.883e-06 -5.550e-06]
     nfev: 6514
     njev: 6514
 hess_inv: <10000x10000 LbfgsInvHessProduct with dtype=float64>
0.005 1e-06
  message: STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
  success: False
   status: 1
      fun: 1.7938831258852321
        x: [ 7.660e+00 -6.943e-01 ... -2.142e+01  3.735e+00]
      nit: 12410
      jac: [ 1.850e-04  7.700e-05 ...  2.071e-04  2.006e-04]
     nfev: 15001
     njev: 15001
 hess_inv: <10000x10000 LbfgsInvHessProduct with dtype=float64>
0.005 0.0001
  message: STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
  success: False
   status: 1
      fun: 47.59912118905323
        x: [ 3.796e+00  5.198e-01 ... -9.192e+00  1.542e+00]
      nit: 12507
      jac: [-2.006e-03 -2.534e-03 ... -3.576e-03 -2.106e

,Gamma,Lambda,Testing Accuracy,Time Taken
0,0.100,0,0.9632,1.493977
1,0.100,0.000001,0.9652,2.246975
2,0.100,0.0001,0.9636,2.448165
3,0.100,0.01,0.9612,1.774225
4,0.050,0,0.9798,2.277664
5,0.050,0.000001,0.9804,7.265285
6,0.050,0.0001,0.9796,10.249683
7,0.050,0.01,0.9766,7.511417
8,0.020,0,0.9818,12.515805
9,0.020,0.000001,0.9844,69.586644


TODO: What was the best test error achieved, and what setting of parameters achieved this error? Was the kernel logistic regression classifier more sensitive to changes in `gamma` or `lamb`? Discuss in 1-3 short sentences below.

##### Discussion

From the table above we can make the following observations:

- The best test-accuracy achieved by KLR was 98.46% for the parameters [$\gamma$ = 0.02, $\lambda$ = 1e-4]
- Test-Accuracy was sensitive with changes in $\gamma$ as it affects the kernal function (RBF)
- Small changes in $\gamma$ had significant influence on performance/accuracy
- $\lambda$ majorly controlled the overfitting and accuracy was affected gradually by it

Now let's do the same thing for the kernel Support Vector Classifier. Train an SVM classifier with **all combinations** of the parameters included below in vectors `gamma` and `C`. For each setting of parameters, compute:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

In [155]:
gammas = [.1, .05,.02,.01,.005]
Cs = [.01,.1,1,10]
# TODO

# ...

dfSVM = pd.DataFrame(columns=['Gamma', 'C', 'Testing Accuracy', 'Time Taken'])

for gamma in gammas:
    for C in Cs:
        svcrbf = svm.SVC(probability=False,  kernel="rbf", C=C, gamma=gamma ,verbose=10)

        startTime = time.time()
        svcrbf.fit(Xtr1, ytr8)
        ysvm = svcrbf.predict(Xts1)
        acc = np.mean(yts8 == ysvm)
        endTime = time.time()
        dfSVM = pd.concat([dfSVM, pd.DataFrame([[gamma, C, acc, endTime - startTime]], columns=dfSVM.columns)], ignore_index=True)

dfSVM

[LibSVM]*.....*
optimization finished, #iter = 5982
obj = -18.769570, rho = -0.997746
nSV = 6918, nBSV = 949
Total nSV = 6918


/var/folders/3y/x88zbv4n0b18f8cg1qcv1my40000gn/T/ipykernel_39334/3016780687.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfSVM = pd.concat([dfSVM, pd.DataFrame([[gamma, C, acc, endTime - startTime]], columns=dfSVM.columns)], ignore_index=True)


[LibSVM].....*....*
optimization finished, #iter = 9470
obj = -177.776034, rho = -0.977664
nSV = 7843, nBSV = 946
Total nSV = 7843
[LibSVM].........*...*
optimization finished, #iter = 12277
obj = -938.426025, rho = -0.808865
nSV = 7968, nBSV = 600
Total nSV = 7968
[LibSVM].........*....*
optimization finished, #iter = 13250
obj = -1011.405531, rho = -0.752041
nSV = 8019, nBSV = 0
Total nSV = 8019
[LibSVM]*.
*
optimization finished, #iter = 1894
obj = -18.444899, rho = -0.995437
nSV = 2761, nBSV = 1426
Total nSV = 2761
[LibSVM].
*.*
optimization finished, #iter = 2624
obj = -146.332194, rho = -0.956396
nSV = 2975, nBSV = 1339
Total nSV = 2975
[LibSVM]...*..*
optimization finished, #iter = 5222
obj = -469.477894, rho = -0.785850
nSV = 3417, nBSV = 231
Total nSV = 3417
[LibSVM]....*.*
optimization finished, #iter = 5745
obj = -491.710632, rho = -0.762837
nSV = 3563, nBSV = 0
Total nSV = 3563
[LibSVM]*.
*
optimization finished, #iter = 1079
obj = -18.019023, rho = -1.023046
nSV = 1957, nB

,Gamma,C,Testing Accuracy,Time Taken
0,0.100,0.01,0.9022,18.640530
1,0.100,0.10,0.9022,29.459618
2,0.100,1.00,0.9282,35.433605
3,0.100,10.00,0.9342,41.849773
4,0.050,0.01,0.9022,8.341913
5,0.050,0.10,0.9228,9.192097
6,0.050,1.00,0.9774,10.357674
7,0.050,10.00,0.9802,10.820482
8,0.020,0.01,0.9022,5.133613
9,0.020,0.10,0.9512,4.564300


TODO: What was the best test error achieved, and what setting of parameters achieved this error? Which performed better in terms of accuracy, the SVM or logisitic regression classifier? How about in terms of runtime? **Add some discussion here.**

##### Discussion

- In Kernal Logistic Regression, we can see from the table that the best test-accuracy of 98.46% was achieved when $\gamma$ = 0.02 and $\lambda$ = 1e-4
- In SVM, we can see from the table that the best test accuracy of 98.7% was achieved when $\gamma$ = 0.01 and $\lambda$ = 10
- In SVM, we can observe that, for the same value of $\gamma$, the test-accuracy was increasing monotonically as the value of $C$ increased

##### Observations
- Overall Kernel Logistic Regression was able to maintain a test accuracy of more than 96% meanwhile SVM had varying test accuracy from 90% to 98.7% depending on the values of $\gamma$ and $\lambda$
- From the column `Time Taken`, we can observe that Kernel Logistic Regression takes significantly more amount of time than SVM, hence states SVM is much faster than Kernel Logistic Regression when the dataset is huge (our test dataset was of size 5000)
- KLR is better for significant smaller datasets

Finally, considering the computational time and accuracy to be achieved for a given scenario, SVM is to be preferred more than KLR because of the observations made above.

**NOTE:** For `sklearns`'s built in classifiers, including svm.SVC, there is a function called `GridSearchCV` which can automatically perform hyperparamater tuning for you. The main advantage of the method (as opposed to writing for-loops) is that it supports parallelization, so it can fit with different parameters at the same time.

In [173]:
from sklearn.model_selection import GridSearchCV

params = {
    'C': [.01,.1,1,10],
    'gamma': [.1, .05,.02,.01,.005],
    'kernel': ['rbf']
}

svcGrid = svm.SVC()

grid_search = GridSearchCV(estimator=svcGrid, param_grid=params, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(Xtr1, ytr8)

print(f"Best Parameters: {grid_search.best_params_}")
best_model = grid_search.best_estimator_
y_pred = best_model.predict(Xts1)
accuracy = np.mean(yts8 == y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END .....................C=0.01, gamma=0.02, kernel=rbf; total time=  11.1s
[CV] END .....................C=0.01, gamma=0.05, kernel=rbf; total time=  17.9s
[CV] END .....................C=0.01, gamma=0.05, kernel=rbf; total time=  17.9s
[CV] END .....................C=0.01, gamma=0.05, kernel=rbf; total time=  18.0s
[CV] END .....................C=0.01, gamma=0.05, kernel=rbf; total time=  18.1s
[CV] END .....................C=0.01, gamma=0.05, kernel=rbf; total time=  18.1s
[CV] END .....................C=0.01, gamma=0.02, kernel=rbf; total time=  10.9s
[CV] END .....................C=0.01, gamma=0.01, kernel=rbf; total time=  10.8s
[CV] END .....................C=0.01, gamma=0.02, kernel=rbf; total time=  11.0s
[CV] END .....................C=0.01, gamma=0.01, kernel=rbf; total time=  10.9s
[CV] END .....................C=0.01, gamma=0.02, kernel=rbf; total time=  11.1s
[CV] END .....................C=0.01, gamma=0.0